# Описание гипотезы
### Проверим предположения
1. Определить, насколько обучение сокращает время прохождения этапов игры.
2. Доказать, что успешное обучение само по себе влияет на оплату и не имеет значения то, каким этапом оно шло.
3. Проверить, насколько прозрачен процесс взаимодействия с игрой.
#### Проверять будем на данных пользователей, зарегистрированных в 2018 году (1 января 2018 - 31 декабря 2018)

Импротируем нужные библиотеки

In [46]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px

Загружаем данные

In [47]:
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')
purchase_df = pd.read_csv('data/purchase.csv', sep=',')

## Производим предобработку данных

Выделяем пользователей зарегистрирванных в 2018 году и загружаем данные по ним в отдельные датафреймы

In [48]:
# Создаем фильтр по пользователям, зарегистрировавшихся в 2018 году
mask = (events_df['start_time'] >= '2018-01-01') & (events_df['start_time'] < '2019-01-01') & (events_df['event_type'] == 'registration')

regist_2018 = events_df[mask]['user_id'].to_list() # список пользователей зарегистрировавшихся в 2018 году 

events_2018 = events_df[events_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице events
purchase_2018 = purchase_df[purchase_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице purchase


Преобразуем даты в полученых датафреймах

In [49]:
purchase_2018['event_datetime'] = pd.to_datetime(purchase_2018['event_datetime'])
events_2018['start_time'] = pd.to_datetime(events_2018['start_time'])

Перед объединением датафреймов, переименуем колонки идентификаторов

In [58]:
events_2018 = events_2018.rename(columns={'id':'event_id'})
purchase_2018 = purchase_2018.rename(columns={'id':'purchase_id'})
purchase_2018['event_type'] = 'purchase'

Сделаем объединенный датафрейм из событий и оплат

In [59]:
total_events_df = pd.concat([events_2018,purchase_2018],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

### Проверим 1-ю гипотезу
* Насколько обучение сокращает время прохождения этапов игры

Сформируем группы пользователей, которые проходили обучение и которые не проходили

In [60]:
# Пользователи которые прошли обучение
users_tutorial_finish = total_events_df[total_events_df['event_type'] == 'tutorial_finish']['user_id'].unique()
# Пользователи которые не проходили обучение
users_not_tutorial = total_events_df[~(total_events_df['user_id'].isin(users_tutorial_finish))]['user_id'].unique()

Разобьем пользователей на группы

In [113]:
total_events_df['groups'] = total_events_df.apply(lambda x: 'С обучением' if x['user_id'] in users_tutorial_finish else 'Без обучения', axis=1)

Сформируем сводную таблицу прохождения этапов игры

In [114]:
users_etap = total_events_df[(total_events_df['event_type'] == 'pack_choice') | (total_events_df['event_type'] == 'level_choice')]

In [115]:
users_etap = users_etap.pivot_table(
    values='start_time',
    index=['user_id','groups'],
    columns='event_type',
    aggfunc='min'
)

Вычислим разницу во времени прохождения этапов игры

In [116]:
users_etap['time_diff'] = users_etap['pack_choice'] - users_etap['level_choice']

Вычислим среднее время прохождения по группам

In [117]:
group_avg = users_etap.groupby(by='groups')[['time_diff']].mean()

Выведим график

In [121]:
fig = px.bar(
    data_frame=group_avg,
    x=group_avg.index,
    y='time_diff',
    color=group_avg.index,
    orientation='v',
    height=500,
    width=500,
    title='Среднее время прохождения этапов игры',
    labels={'time_diff':'Время прохождения', 'groups':'Группы'}
)
fig.update_yaxes(range=[0,6], selector='min')
fig.show()

In [64]:
total_events_df.event_type.unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice', 'purchase'], dtype=object)

In [89]:
users_etap

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount,groups
51424,80327.0,level_choice,medium,2018-01-01 20:37:22,NaN,27835,NaN,NaT,NaN,1
51425,80328.0,pack_choice,NaN,2018-01-01 20:38:43,NaN,27835,NaN,NaT,NaN,1
51428,80331.0,level_choice,hard,2018-01-01 22:37:50,NaN,27839,NaN,NaT,NaN,1
51429,80332.0,pack_choice,NaN,2018-01-01 22:42:54,NaN,27839,NaN,NaT,NaN,1
51441,80344.0,level_choice,medium,2018-01-02 05:18:42,NaN,27840,NaN,NaT,NaN,0
...,...,...,...,...,...,...,...,...,...,...
118353,147256.0,level_choice,medium,2018-12-31 22:36:19,NaN,47753,NaN,NaT,NaN,1
118354,147257.0,pack_choice,NaN,2018-12-31 22:38:05,NaN,47753,NaN,NaT,NaN,1
118362,147265.0,level_choice,easy,2019-01-01 05:04:52,NaN,47755,NaN,NaT,NaN,1
118363,147266.0,level_choice,medium,2019-01-01 05:42:11,NaN,47756,NaN,NaT,NaN,1


In [93]:
group_avg.columns

Index(['time_diff'], dtype='object', name='event_type')

In [28]:
level_choise_total

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
19,80327.0,level_choice,medium,2018-01-01 20:37:22,NaN,27835,NaN,NaT,NaN
23,80331.0,level_choice,hard,2018-01-01 22:37:50,NaN,27839,NaN,NaT,NaN
36,80344.0,level_choice,medium,2018-01-02 05:18:42,NaN,27840,NaN,NaT,NaN
38,80346.0,level_choice,hard,2018-01-02 06:19:18,NaN,27845,NaN,NaT,NaN
45,80353.0,level_choice,easy,2018-01-02 08:46:03,NaN,27842,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
66932,147240.0,level_choice,medium,2018-12-31 09:59:00,NaN,47732,NaN,NaT,NaN
66937,147245.0,level_choice,medium,2018-12-31 12:24:40,NaN,47747,NaN,NaT,NaN
66948,147256.0,level_choice,medium,2018-12-31 22:36:19,NaN,47753,NaN,NaT,NaN
66956,147265.0,level_choice,easy,2019-01-01 05:04:52,NaN,47755,NaN,NaT,NaN
